# Full Pipeline - Insurance Policy Generation Chatbot

In retrieval augmented generation (RAG), an LLM retrieves contextual documents from an external dataset as part of its execution.

This processs is illustrated in the figure below.

![pipeline](assets/pipeline.jpeg)

from the Document Loading phase to the Output phase, the pipeline is as follows:

1. **Document Loading**: The documents are loaded from the external dataset.
2. **Splitting**: The documents are split into chunks.
3. **Storage**: The chunks are stored in a database.
4. **Retrieval**: The LLM retrieves the relevant chunks from the database.
5. **Agent Output & Testing**: The Conversational Retrieval Agent outputs the retrieved chunks to the user. Moreover, the agent is tested on a set of test questions.


## Setup & Initialization:

Let's Import dependencies and initialize some variables.


In [12]:
%pip install -r requirements.txt >> /dev/null

Note: you may need to restart the kernel to use updated packages.


In [1]:
%load_ext autoreload
%autoreload 2

import glob
import os

# import openai api and set api key
import openai

# import src modules
from src import config
from src import etl
from src.custom_qa_generate_chain import CustomQAGenerateChain

# import langchain related modules
from chromadb.config import Settings
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.llms import OpenAI
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.agents import Tool
from langchain.utilities import GoogleSearchAPIWrapper
from langchain.agents.openai_functions_agent.agent_token_buffer_memory import (
    AgentTokenBufferMemory,
)
from langchain.agents.agent_toolkits import create_retriever_tool
from langchain.agents.openai_functions_agent.base import OpenAIFunctionsAgent
from langchain.schema.messages import SystemMessage
from langchain.prompts import MessagesPlaceholder
from langchain.agents import AgentExecutor
from langchain.evaluation.qa import QAEvalChain
from langchain.output_parsers.regex import RegexParser


openai.api_key = config.OPENAI_API_KEY

## Document Loading:

We're going to skip all the explanation and jump straight into the code. If you want to know more about the document exploratory analysis, please refer to the [explore_dataset.ipynb](explore_dataset.ipynb) notebook.


In [2]:
loader = PyPDFDirectoryLoader(config.DATASET_ROOT_PATH)
documents = loader.load()

In [3]:
documents = etl.load_documents_with_title(config.DATASET_ROOT_PATH)

In [4]:
documents = etl.preprocess(documents)

Now that we have preprocessed the data, we can proceed to build our LLM.


## Document Splitting

Now, let's split the document into chunks of text for further processing. How we decided which technique to use is explained in the [explore_dataset.ipynb](explore_dataset.ipynb) notebook.


In [5]:
r_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=150,
    length_function=len,
    strip_whitespace=True,
)

r_splitter_result = r_splitter.split_documents(documents)

print("Chunk Size count: ", len(r_splitter_result))

print(r_splitter_result[0].page_content)
print("-------------------")
print(r_splitter_result[1].page_content)
print("-------------------")
print(r_splitter_result[2].page_content)

Chunk Size count:  770
SEGURO COLECTIVO COMPLEMENTARIO DE SALUD 
Incorporada al Depósito de Pólizas bajo el código POL320130223
ARTICULO 1°: REGLAS APLICABLES AL CONTRATO

Se aplicarán al presente contrato de seguro las disposiciones contenidas en los artículos siguientes y las
normas legales de carácter imperativo establecidas en el Título VIII, del Libro II, del Código de Comercio. Sin
embargo, se entenderán válidas las estipulaciones contractuales que sean más beneficiosas para el
asegurado o beneficiario.

ARTÍCULO Nº 2: COBERTURA

La compañía de seguros bajo las condiciones y términos que más adelante se establecen, conviene en
reembolsar o pagar al beneficiario, los gastos médicos razonables y acostumbrados en que haya incurrido
efectivamente un asegurado, en complemento de lo que cubra el sistema de salud previsional o de bienestar
u otro seguro o convenio, a consecuencia de una incapacidad cubierta.
-------------------
Se otorgará cobertura a los gastos médicos incurridos por l

## Storage

Now that we have our chunks, we can feed them to our vector store. But first, we need to convert them into embeddings.

### Embedding

We will use the OpenAI GPT-3.5 Turbo model to generate embeddings for our chunks.


In [6]:
embedding = OpenAIEmbeddings()

### Vector Store

We will use Chroma to store our embeddings. Chroma is a vector store that allows us to store and query embeddings.


In [7]:
if not glob.glob(os.path.join(config.CHROMA_PATH, "*.sqlite3")):
    print("Creating new Chroma DB")
    settings = Settings(
        chroma_db_impl="duckdb+parquet",
        persist_directory=config.CHROMA_PATH,
        anonymized_telemetry=False,
    )
    vector_store = Chroma.from_documents(
        r_splitter_result, embedding, persist_directory=config.CHROMA_PATH
    )
else:
    print("Loading Chroma DB")
    vector_store = Chroma(
        persist_directory=config.CHROMA_PATH, embedding_function=embedding
    )

Creating new Chroma DB


Let's see the vector count.


In [8]:
print(vector_store._collection.count())

770


### Testing Similarity Search


Now that we have our vector store, let's test it by performing a similarity search on a query.


In [9]:
question = "Cuales son algunas de las limitaciones de la cobertura?"

Here, we retrieve the top 2 most similar chunks to our query.


In [10]:
response_ss = vector_store.similarity_search(question, k=5)

Let's see how many chunks we have retrieved.


In [11]:
len(response_ss)

5

Now, we will print the top 2 most similar chunks to our query.


In [12]:
for page in response_ss:
    print("The Page's File name is: ", page.metadata["source"])
    print("The Page number is: ", page.metadata["page"])
    print(page.page_content)
    print("-------------------")

The Page's File name is:  /home/gio/ANYONEAI/InsurancePolicyChatbot/notebook/dataset/POL320130223.pdf
The Page number is:  14
LIMITACIONES DE LAS COBERTURAS:

Sin perjuicio de los porcentajes y límites de reembolso o pago que puedan establecerse en las condiciones
particulares, la presente póliza contempla las siguientes limitaciones de cobertura:

1. En aquellos casos en que el asegurado no esté afiliado a un sistema de salud previsional, privado o
estatal, se considerará como gasto efectivamente incurrido el monto que resulte de la aplicación del
porcentaje que se señala en las Condiciones Particulares de la póliza sobre el gasto médico reclamado.
Sobre el monto resultante se aplicarán los porcentajes y límites de reembolso o pago definidos para cada
cobertura en el Cuadro de Beneficios de las Condiciones Particulares de la póliza.
-------------------
The Page's File name is:  /home/gio/ANYONEAI/InsurancePolicyChatbot/notebook/dataset/POL320190074.pdf
The Page number is:  29
LIMITACI

Let's persist the vector store so we can use it later.


In [13]:
vector_store.persist()

### Failure modes

As you've seen in the previous section, the results are not perfect. Notice that we're getting duplicate information comming from different documents. We will start applying some retrieval strategies to improve the results.


## Retrieval

This section is the centerpiece of our Retrieval Augmented Generation (RAG) Flow and it's where we will apply different strategies to improve the results of our similarity search.


### Addressing Diversity: Maximum marginal relevance

This would be our first attempt to improve the diversity in the search results. `Maximum marginal relevance` (MMR) strives to achieve both relevance to the query and diversity among the results.


In [26]:
response_mmr = vector_store.max_marginal_relevance_search(
    question, k=5, fetch_k=3, lambda_=0.5
)

In [27]:
for page in response_mmr:
    print("The Page's File name is: ", page.metadata["source"])
    print("The Page number is: ", page.metadata["page"])
    print("The Policy's Title is: ", page.metadata["title"])
    print("")
    print(page.page_content)
    print("-------------------")

The Page's File name is:  /home/gio/ANYONEAI/InsurancePolicyChatbot/dataset/POL320190074.pdf
The Page number is:  29
The Policy's Title is:  SEGURO PARA PRESTACIONES MÉDICAS DE ALTO COSTO

LIMITACIONES DE LAS COBERTURAS:

Sin perjuicio de los porcentajes y límites de reembolso o pago que puedan establecerse en las condiciones
particulares, la presente póliza contempla las siguientes limitaciones de cobertura:

1. En aquellos casos en que el asegurado no esté afiliado a un sistema de salud previsional, privado o
estatal, se considerará como gasto efectivamente incurrido el monto que resulte de la aplicación del
porcentaje que se señala en las Condiciones Particulares de la póliza sobre el gasto médico reclamado.
Sobre el monto resultante se aplicarán los porcentajes y límites de reembolso o pago definidos para cada
cobertura en el Cuadro de Beneficios de las Condiciones Particulares de la póliza.
-------------------
The Page's File name is:  /home/gio/ANYONEAI/InsurancePolicyChatbot/dat

### Addressing Specificity: working with metadata using self-query retrievers


What if there is a question related to a specific document? For example, "Cual es la cobertura en SEGURO PARA PRESTACIONES MÉDICAS DE ALTO COSTO?".

Fortunatelly, crhomaDb supports operations on `metadata`.

`metadata` provides context for each embedded chunk.

However, there is an interesting challenge here: we often want to infer the metadata from the query itself.

To address this, we can use `SelfQueryRetriever`, which uses an LLM to extract:

1. The `query` string to use for vector search
2. A metadata filter to pass in as well

Most vector databases support metadata filters, so this doesn't require any new databases or indexes.


In [28]:
metadata_field_info = [
    AttributeInfo(
        name="source",
        type="string",
        description="el nombre de archivo y codigo de la poliza de donde vino este fragmento, el formato es POL{codigo de poliza}.pdf",
    ),
    AttributeInfo(
        name="page", type="integer", description="El numero de pagina de la poliza"
    ),
    AttributeInfo(name="title", type="string", description="El titulo de la poliza"),
]

In [29]:
document_content_description = "Polizas de Seguro"
llm = OpenAI(temperature=config.TEMPERATURE)
retriever_sqr = SelfQueryRetriever.from_llm(
    llm, vector_store, document_content_description, metadata_field_info, verbose=True
)

In [30]:
question = "Cual es la cobertura en la poliza PÓLIZA DE ACCIDENTES PERSONALES / REEMBOLSO GASTOS MÉDICOS?"

In [31]:
retriever_results = retriever_sqr.get_relevant_documents(question)

/home/gio/ANYONEAI/InsurancePolicyChatbot/.venv/lib/python3.10/site-packages/langchain/chains/llm.py:280: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


query='cobertura poliza accidentes personales reembolso gastos medicos' filter=Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='title', value='PÓLIZA DE ACCIDENTES PERSONALES / REEMBOLSO GASTOS MÉDICOS') limit=None


In [32]:
for k in retriever_results:
    print("The Page's File name is: ", k.metadata["source"])
    print("The Page number is: ", k.metadata["page"])
    print("The Policy's Title is: ", k.metadata["title"])
    print("")
    print(k.page_content)

The Page's File name is:  /home/gio/ANYONEAI/InsurancePolicyChatbot/dataset/POL120190177.pdf
The Page number is:  0
The Policy's Title is:  PÓLIZA DE ACCIDENTES PERSONALES / REEMBOLSO GASTOS MÉDICOS

PÓLIZA DE ACCIDENTES PERSONALES / REEMBOLSO GASTOS MÉDICOS
Incorporada al Depósito de Pólizas bajo el código POL120190177
ARTÍCULO 1°: REGLAS APLICABLES AL CONTRATO
Se aplicarán al presente contrato de seguro las disposiciones contenidas en los artículos siguientes y las
normas legales de carácter imperativo establecidas en el título VIII, del Libro II, del Código de Comercio. Sin
embargo, se entenderán válidas las estipulaciones contractuales que sean más beneficiosas para el
asegurado o el beneficiario.
ARTÍCULO 2º: COBERTURA Y MATERIA ASEGURADA
La Compañía Aseguradora reembolsará al asegurado o pagará directamente al prestador de salud los
Gastos Médicos Razonables y Acostumbrados y Efectivamente Incurridos, una vez se haya otorgado y
pagado la cobertura del sistema de salud previsional

Now we can see that the results are more specific to the query. However, this can be improved even further.


### Last Trick: Compression

Another approach for improving the quality of retrieved docs is compression.

Information most relevant to a query may be buried in a document with a lot of irrelevant text.

Passing that full document through your application can lead to more expensive LLM calls and poorer responses.

Contextual compression is meant to fix this. In this particular case, we will combine the `ContextualCompressionRetriever` with the `MMR` retriever to improve the quality of our results.


In [33]:
compressor = LLMChainExtractor.from_llm(llm)

In [34]:
compressor_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=vector_store.as_retriever(search_type="mmr"),
)

In [35]:
question = "de que habla el Mantenimiento artificial de la vida?"

In [36]:
compressed_documents = compressor_retriever.get_relevant_documents(question)
etl.pretty_print_docs(compressed_documents)

Document 1:

"MANTENIMIENTO ARTIFICIAL DE LA VIDA: tratamiento médico que sustituye o mantiene activas las funciones vitales del cuerpo, que temporal o permanentemente no pueden realizarse de forma independiente, e incluye la respiración asistida, la nutrición e hidratación artificiales y la reanimación cardiopulmonar."
----------------------------------------------------------------------------------------------------
Document 2:

"Mantenimiento artificial de la vida: el Asegurador no pagará el mantenimiento artificial de la vida cuando el paciente sufra de una lesión, enfermedad o padecimiento que requiera tratamiento para el mantenimiento artificial de la vida, cuando no se espere que dichos tratamientos resulten en la recuperación del asegurado"
----------------------------------------------------------------------------------------------------
Document 3:

ii. Estudios y tratamientos por talla baja, gigantismo y todo tipo de hormonas del crecimiento, así como también Antagonistas 

Now, the documents are much more short and specific to the query.


## Question Answering

Now that we have our retriever, we can use it to answer questions. In this section, we will explore 3 different approaches to question answering:


This is the template we will use for our question answering pipeline:

```python

Eres un asistente bien informado centrado en pólizas de seguro y documentos. Utilizando el contexto proporcionado de nuestra base de datos de polizas de seguros, responde a la siguiente pregunta relacionada con seguros. Asegúrate de proporcionar sólo información relevante a las pólizas de seguro y documentos y evita responder a preguntas no relacionadas con este dominio.

Utiliza las siguientes piezas de contexto para responder a la pregunta al final. Si no sabes la respuesta, simplemente di que no lo sabes, no intentes inventar una respuesta. Usa un máximo de tres frases. Mantén la respuesta lo más concisa posible. ¡Siempre di "¡gracias por preguntar!" al final de la respuesta!
{contexto}
Pregunta: {pregunta}
Respuesta útil:"""

```


Let's set up our LLM Agent. We will use `gp3-5-turbo` for this task.


In [37]:
llm = ChatOpenAI(model_name=config.FAST_LLM_MODEL, temperature=config.TEMPERATURE)

In [38]:
question = "Cuales las limitaciones de la cobertura de la poliza PÓLIZA DE ACCIDENTES PERSONALES / REEMBOLSO GASTOS MÉDICOS?"

In [39]:
template_prompt = """
Eres un asistente bien informado centrado en pólizas de seguro y documentos. Utilizando el contexto proporcionado de nuestra base de datos de polizas de seguros, responde a la siguiente pregunta relacionada con seguros. Asegúrate de proporcionar sólo información relevante a las pólizas de seguro y documentos y evita responder a preguntas no relacionadas con este dominio. 

Utiliza las siguientes piezas de contexto para responder a la pregunta al final. Si no sabes la respuesta, simplemente di que no lo sabes, no intentes inventar una respuesta. Usa un máximo de tres frases. Mantén la respuesta lo más concisa posible. ¡Siempre di "¡gracias por preguntar!" al final de la respuesta!
{context}
Pregunta: {question}
Respuesta útil:"""

QA_CHAIN_PROMPT = PromptTemplate(
    template=template_prompt, input_variables=["context", "question"]
)

In [40]:
qa_chain_mr = RetrievalQA.from_chain_type(
    llm,
    retriever=retriever_sqr,
    chain_type="stuff",
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT},
)

In [41]:
results = qa_chain_mr({"query": question})

query='limitaciones cobertura poliza' filter=Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='title', value='PÓLIZA DE ACCIDENTES PERSONALES / REEMBOLSO GASTOS MÉDICOS') limit=None


In [42]:
print(results["result"])

Las limitaciones de cobertura de la póliza de Accidentes Personales / Reembolso Gastos Médicos incluyen: 
- En caso de que el asegurado no esté afiliado a un sistema de salud previsional, privado o estatal, la compañía calculará los gastos a ser reembolsados o pagados al Prestador y aplicará el porcentaje de reembolso o pago al prestador establecido en el Cuadro de Coberturas de las Condiciones Particulares de la Póliza.
- En caso de que el asegurado esté afiliado a un sistema de salud previsional, privado o estatal, y las prestaciones o gastos reclamados no cuenten con bonificación, aporte y/o reembolso mayor que cero (0) pesos según el plan o contrato de salud previsional contratado, la cobertura será limitada.
- El pago al prestador o reembolso al asegurado estará limitado por el monto definido en el Arancel del Prestador indicado en las condiciones particulares de la póliza.

¡Gracias por preguntar!


Let's see the source of the answer.


In [43]:
print(results["source_documents"][0].page_content)
print("-------------------")
print(results["source_documents"][1].page_content)

lesiones.
ARTÍCULO 3°: LIMITACIONES DE LAS COBERTURAS
Sin perjuicio de los términos, porcentajes, límites y topes de reembolso que se establezcan en el Cuadro de
Coberturas de las Condiciones Particulares de la Póliza. Ésta última contempla las siguientes limitaciones de
cobertura:
a) En aquellos casos en que el asegurado no esté afiliado a un sistema de salud previsional, privado o
estatal, la compañía calculará los gastos a ser reembolsados o pagados al Prestador y sobre ellos aplicará el
porcentaje de reembolso o pago al prestador que se indica para estos efectos en el Cuadro de Coberturas
de las Condiciones Particulares de la Póliza.
b) En aquellos casos en que el asegurado esté afiliado a un sistema de salud previsional, privado o estatal, y
las prestaciones o gastos reclamados no cuenten con bonificación, aporte y/o reembolso mayor que cero (0)
pesos en conformidad al plan o contrato de salud previsional contratado en tales instituciones, por cualquier
-------------------
efectos

### RetrievalQA limitations

QA fails to preserve conversational history. This means that if we ask a follow-up question, the answer will be based on the original question, not the follow-up question. This is because the QA model is not aware of the previous question and answer. We need to implement memory to address this issue.


## Addressing Conversational Memory

In order to address the conversational memory issue, we will use a memory module that will store the previous questions and answers. This will allow us to use the previous question and answer as context for the next question. For this, we will be using `ConversationalBufferMemory` from `langchain.memory`.


In [44]:
memory_key = "chat_history"

memory = ConversationBufferMemory(
    memory_key=memory_key,
    return_messages=True,
    output_key="answer",
    input_key="question",
)

Now we need to make our retriever work together with our memory. For this we need to use `ConversationalRetrievalChain` in where we pass our language model, our retriever, and our memory. The conversational retriever adds a new bit on top of the retrieval QA Chain, not just memory.

Specifically, what it adds it's a step that takes the history and the new question and condenses it into a stand-alone question to pass to the vector store to look up for relevant documents.

We need to redo our prompt to support memory.


In [45]:
template_prompt_with_memory = """
Eres un asistente bien informado centrado en pólizas de seguro y documentos. Utilizando el contexto proporcionado de nuestra base de datos de polizas de seguros, responde a la siguiente pregunta relacionada con seguros. Asegúrate de proporcionar sólo información relevante a las pólizas de seguro y documentos y evita responder a preguntas no relacionadas con este dominio. 

Dado el siguiente historial de conversación:

{chat_history}

Utiliza las siguientes piezas de contexto e historial para responder a la pregunta al final. Si no sabes la respuesta, simplemente di que no lo sabes, no intentes inventar una respuesta. Usa un máximo de tres frases. Mantén la respuesta lo más concisa posible. ¡Siempre di "¡gracias por preguntar!" al final de la respuesta!
{context}
Pregunta: {question}
Respuesta útil:"""

TEMPLATE_PROMPT_WITH_MEMORY = PromptTemplate(
    template=template_prompt_with_memory,
    input_variables=["context", "question", "chat_history"],
)

In [46]:
qa = ConversationalRetrievalChain.from_llm(
    llm,
    retriever=retriever_sqr,
    chain_type="stuff",
    memory=memory,
    get_chat_history=lambda h: h,
    verbose=True,
    combine_docs_chain_kwargs={"prompt": TEMPLATE_PROMPT_WITH_MEMORY},
)

In [47]:
question = "Cuales las limitaciones de la cobertura de la poliza PÓLIZA DE ACCIDENTES PERSONALES / REEMBOLSO GASTOS MÉDICOS?"

result = qa({"question": question})

query='limitaciones cobertura poliza' filter=Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='title', value='PÓLIZA DE ACCIDENTES PERSONALES / REEMBOLSO GASTOS MÉDICOS') limit=None


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Eres un asistente bien informado centrado en pólizas de seguro y documentos. Utilizando el contexto proporcionado de nuestra base de datos de polizas de seguros, responde a la siguiente pregunta relacionada con seguros. Asegúrate de proporcionar sólo información relevante a las pólizas de seguro y documentos y evita responder a preguntas no relacionadas con este dominio. 

Dado el siguiente historial de conversación:

[]

Utiliza las siguientes piezas de contexto e historial para responder a la pregunta al final. Si no sabes la respuesta, simplemente di que no lo sabes, no intentes inventar una respuesta. Usa un máximo de tres frases. Mantén la respuesta lo más concisa posible. ¡Siempre di "¡grac

In [48]:
print(result["answer"])

Las limitaciones de cobertura de la póliza de Accidentes Personales / Reembolso Gastos Médicos incluyen: 
- No cubre gastos médicos que no tengan como causa un evento.
- No cubre gastos que provengan, se originen o sean consecuencia de complicaciones de lesiones preexistentes, tratamientos médicos quirúrgicos distintos a los necesarios por lesiones cubiertas, catástrofes naturales y cirugías o tratamientos estéticos, cosméticos, plásticos, reparadores, maxilofaciales, ortopédicos y otros. 
¡Gracias por preguntar!


In [49]:
question = "que tipo de gastos medicos has dicho que no cubre?"
result = qa({"question": question})



> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
[HumanMessage(content='Cuales las limitaciones de la cobertura de la poliza PÓLIZA DE ACCIDENTES PERSONALES / REEMBOLSO GASTOS MÉDICOS?', additional_kwargs={}, example=False), AIMessage(content='Las limitaciones de cobertura de la póliza de Accidentes Personales / Reembolso Gastos Médicos incluyen: \n- No cubre gastos médicos que no tengan como causa un evento.\n- No cubre gastos que provengan, se originen o sean consecuencia de complicaciones de lesiones preexistentes, tratamientos médicos quirúrgicos distintos a los necesarios por lesiones cubiertas, catástrofes naturales y cirugías o tratamientos estéticos, cosméticos, plásticos, reparadores, maxilofaciales, ortopédicos y otros. \n¡Gracias por preguntar!', additional_kwargs={}, example=False)]
Follow Up Input: que t


> Finished chain.
query='gastos médicos no cubre' filter=None limit=None


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Eres un asistente bien informado centrado en pólizas de seguro y documentos. Utilizando el contexto proporcionado de nuestra base de datos de polizas de seguros, responde a la siguiente pregunta relacionada con seguros. Asegúrate de proporcionar sólo información relevante a las pólizas de seguro y documentos y evita responder a preguntas no relacionadas con este dominio. 

Dado el siguiente historial de conversación:

[HumanMessage(content='Cuales las limitaciones de la cobertura de la poliza PÓLIZA DE ACCIDENTES PERSONALES / REEMBOLSO GASTOS MÉDICOS?', additional_kwargs={}, example=False), AIMessage(content='Las limitaciones de cobertura de la póliza de Accidentes Personales / Reembolso Gastos Médicos incluyen: \n- No cubre gastos médicos que no tengan como causa un evento.\n- No cubre gastos que provengan,

In [50]:
print(result["answer"])

Las limitaciones de cobertura de la póliza de Accidentes Personales / Reembolso Gastos Médicos incluyen gastos médicos que no tengan como causa un evento, gastos que provengan de complicaciones de lesiones preexistentes, tratamientos médicos quirúrgicos distintos a los necesarios por lesiones cubiertas, catástrofes naturales y cirugías o tratamientos estéticos, cosméticos, plásticos, reparadores, maxilofaciales, ortopédicos y otros. ¡Gracias por preguntar!


In [51]:
question = (
    "dame un resumen de la cobertura del SEGURO PARA PRESTACIONES MÉDICAS DE ALTO COSTO"
)
result = qa({"question": question})



> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
[HumanMessage(content='Cuales las limitaciones de la cobertura de la poliza PÓLIZA DE ACCIDENTES PERSONALES / REEMBOLSO GASTOS MÉDICOS?', additional_kwargs={}, example=False), AIMessage(content='Las limitaciones de cobertura de la póliza de Accidentes Personales / Reembolso Gastos Médicos incluyen: \n- No cubre gastos médicos que no tengan como causa un evento.\n- No cubre gastos que provengan, se originen o sean consecuencia de complicaciones de lesiones preexistentes, tratamientos médicos quirúrgicos distintos a los necesarios por lesiones cubiertas, catástrofes naturales y cirugías o tratamientos estéticos, cosméticos, plásticos, reparadores, maxilofaciales, ortopédicos y otros. \n¡Gracias por preguntar!', additional_kwargs={}, example=False), HumanMessage(content='


> Finished chain.
query='SEGURO PARA PRESTACIONES MÉDICAS DE ALTO COSTO' filter=None limit=None


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Eres un asistente bien informado centrado en pólizas de seguro y documentos. Utilizando el contexto proporcionado de nuestra base de datos de polizas de seguros, responde a la siguiente pregunta relacionada con seguros. Asegúrate de proporcionar sólo información relevante a las pólizas de seguro y documentos y evita responder a preguntas no relacionadas con este dominio. 

Dado el siguiente historial de conversación:

[HumanMessage(content='Cuales las limitaciones de la cobertura de la poliza PÓLIZA DE ACCIDENTES PERSONALES / REEMBOLSO GASTOS MÉDICOS?', additional_kwargs={}, example=False), AIMessage(content='Las limitaciones de cobertura de la póliza de Accidentes Personales / Reembolso Gastos Médicos incluyen: \n- No cubre gastos médicos que no tengan como causa un evento.\n- No cubr

In [52]:
print(result["answer"])

El SEGURO PARA PRESTACIONES MÉDICAS DE ALTO COSTO cubre los Gastos Médicos Razonables y Acostumbrados y Efectivamente Incurridos una vez otorgada y pagada la cobertura del sistema de salud previsional, seguros complementarios u otros beneficios contratados por el asegurado. Esto incluye enfermedades diagnosticadas por primera vez, accidentes que requieran internación hospitalaria y gastos médicos complementarios a los cubiertos por otros sistemas de salud. ¡Gracias por preguntar!


In [53]:
question = "osea que el seguro para prestaciones medicas de alto costo cubre gastos medicos una vez otorgada y pagada la cobertura del sistema de salud?"
result = qa({"question": question})



> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
[HumanMessage(content='Cuales las limitaciones de la cobertura de la poliza PÓLIZA DE ACCIDENTES PERSONALES / REEMBOLSO GASTOS MÉDICOS?', additional_kwargs={}, example=False), AIMessage(content='Las limitaciones de cobertura de la póliza de Accidentes Personales / Reembolso Gastos Médicos incluyen: \n- No cubre gastos médicos que no tengan como causa un evento.\n- No cubre gastos que provengan, se originen o sean consecuencia de complicaciones de lesiones preexistentes, tratamientos médicos quirúrgicos distintos a los necesarios por lesiones cubiertas, catástrofes naturales y cirugías o tratamientos estéticos, cosméticos, plásticos, reparadores, maxilofaciales, ortopédicos y otros. \n¡Gracias por preguntar!', additional_kwargs={}, example=False), HumanMessage(content='


> Finished chain.
query='prestaciones médicas de alto costo gastos médicos cobertura sistema salud' filter=None limit=None


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Eres un asistente bien informado centrado en pólizas de seguro y documentos. Utilizando el contexto proporcionado de nuestra base de datos de polizas de seguros, responde a la siguiente pregunta relacionada con seguros. Asegúrate de proporcionar sólo información relevante a las pólizas de seguro y documentos y evita responder a preguntas no relacionadas con este dominio. 

Dado el siguiente historial de conversación:

[HumanMessage(content='Cuales las limitaciones de la cobertura de la poliza PÓLIZA DE ACCIDENTES PERSONALES / REEMBOLSO GASTOS MÉDICOS?', additional_kwargs={}, example=False), AIMessage(content='Las limitaciones de cobertura de la póliza de Accidentes Personales / Reembolso Gastos Médicos incluyen: \n- No cubre gastos médicos que no tengan como 

In [54]:
print(result["answer"])

Sí, el seguro para prestaciones médicas de alto costo cubre los gastos médicos una vez otorgada y pagada la cobertura del sistema de salud. ¡Gracias por preguntar!


## Supercharging our chatbot with Google Search


Now our RAG is able to efficiently retrieve documents based on the previous question and answer. Moreover, it is able to answer follow-up questions based on the previous question and answer thanks to the memory module. However, we're not done yet! We need to add one more feature to our chatbot: being able to search the web for answers that are not in our dataset or for questions that specifically ask for information that is not in our dataset related to insurance policies.


#### Get Search Results

We must set up the API Key and a custom search engine to be able to use Google search API. We will proceed to use the LangChain library that provides the `GoogleSearchAPIWrapper` utility that takes care of receiving search results and makes a function to run it `top_n_results`. Then, the `Tool` class will create a wrapper around the said function to make it compatible with agents and help them to interact with the ouside world. We will only process top 3 results and concatenate the result of each query in the `all_results` variable.

We’re going to declare some wrappers. The `GoogleSearchAPIWrapper` wrapper allows us to easily create a tool for using the Google Search APIs


In [55]:
search = GoogleSearchAPIWrapper(
    google_api_key=config.GOOGLE_API_KEY, google_cse_id=config.CUSTOM_SEARCH_ENGINE_ID
)

## Conversational Retrieval Agent - Creating from components

Now, our **QAretriever** will be a tool for our **Conversational Retrieval Agent** just like our search tool. That way our Agent can interact with the QARetriever and be able to use both the vector store to retrieve documents or the Google Search API to retrieve relevant information.

There are a few components:

- Tools ( DocumentRetriever, SearchTool )
- The memory
- The prompt template
- The agent
- The agent executor


To recap, here we have our toolkit set assembled of:

1. The `google-search` tool is a convenient way to perform Google searches when an agent needs information about current events. The tool makes use of Google's API to provide relevant search results.
2. The `retriever` tool is a convenient way to perform searches on a vector store when an agent needs information about insurance policies. The tool makes use of the vector store to provide relevant search results.


In [56]:
toolkit = [
    create_retriever_tool(
        retriever=retriever_sqr,
        name="retriever",
        description="""Util para cuando necesitas buscar en la base de datos de polizas de seguro para responder preguntas""",
    ),
    Tool(
        name="google_search",
        description="""Util para cuando necesitas buscar en internet para responder preguntas acerca de noticias o informacion
        relevante a las polizas de seguro en general que no se encuentran en la base de datos de polizas de seguro""",
        func=search.run,
    ),
]

These tools are then added to the toolkit list, which is used to initialize an agent with the specified tools. The agent can then perform various tasks using the tools in its toolkit. The agent can be easily extended by adding more tools to the toolkit, allowing it to handle a wide range of tasks and situations.


### Memory Tweak

We will redo and redefine our memory to adapt it to our new agent.


In [143]:
memory_key = "chat_history"

memory = AgentTokenBufferMemory(memory_key=memory_key, llm=llm, max_token_limit=2500)

### The Prompt Template

For the prompt template, we will use the `OpenAIFunctionsAgent` default way of creating one, but pass in a system prompt and a placeholder for memory. We're going to reuse our old prompt template and tweak it a little bit.


In [154]:
system_message = SystemMessage(
    content=(
        """Eres un asistente bien informado centrado en pólizas de seguro y documentos. 
        Utilizando el contexto proporcionado de nuestra base de datos de polizas de seguros, responde a la siguiente pregunta relacionada con seguros.
        Asegúrate de proporcionar sólo información relevante a las pólizas de seguro y documentos y evita responder a preguntas no relacionadas con este dominio. 
        Sientete libre de utilizar las tu herramienta de "retriever" para buscar informacion relevante y responder a la pregunta al final.
        Solo puedes usar tu herramienta de "google_search" si te lo pide el usuario explicitamente diciendo "busca en google". No lo hagas si no te lo pide explicitamente.
        Si no sabes la respuesta, simplemente di que no lo sabes, no intentes inventar una respuesta.
        Mantén la respuesta lo más concisa posible.
        
        Dado el siguiente historial de conversación:
        {chat_history}
        
        Pregunta: {input}
        Respuesta útil:"""
    )
)

In [155]:
prompt = OpenAIFunctionsAgent.create_prompt(
    system_message=system_message,
    extra_prompt_messages=[MessagesPlaceholder(variable_name=memory_key)],
)

### The Agent

We will use the `OpenAIFunctionsAgent`.


In [156]:
agent = OpenAIFunctionsAgent(llm=llm, tools=toolkit, prompt=prompt)

### The Agent Executor

This is important, we pass in `return_intermediate_steps=True` to the agent executor since we are recording that with our memory object.


In [157]:
agent_executor = AgentExecutor(
    agent=agent,
    tools=toolkit,
    memory=memory,
    verbose=True,
    return_intermediate_steps=True,
)

Let's test our agent with a question that is in our dataset.


In [119]:
result = agent_executor(
    {
        "input": "Hola, como estas? me podrias ayudar diciendome cuales son las limitaciones de la cobertura de la poliza PÓLIZA DE ACCIDENTES PERSONALES / REEMBOLSO GASTOS MÉDICOS?"
    }
)



> Entering new AgentExecutor chain...

Invoking: `retriever` with `PÓLIZA DE ACCIDENTES PERSONALES / REEMBOLSO GASTOS MÉDICOS`


query='PÓLIZA DE ACCIDENTES PERSONALES / REEMBOLSO GASTOS MÉDICOS' filter=None limit=None
[Document(page_content='PÓLIZA DE ACCIDENTES PERSONALES / REEMBOLSO GASTOS MÉDICOS\nIncorporada al Depósito de Pólizas bajo el código POL120190177\nARTÍCULO 1°: REGLAS APLICABLES AL CONTRATO\nSe aplicarán al presente contrato de seguro las disposiciones contenidas en los artículos siguientes y las\nnormas legales de carácter imperativo establecidas en el título VIII, del Libro II, del Código de Comercio. Sin\nembargo, se entenderán válidas las estipulaciones contractuales que sean más beneficiosas para el\nasegurado o el beneficiario.\nARTÍCULO 2º: COBERTURA Y MATERIA ASEGURADA\nLa Compañía Aseguradora reembolsará al asegurado o pagará directamente al prestador de salud los\nGastos Médicos Razonables y Acostumbrados y Efectivamente Incurridos, una vez se haya otorgado 

In [120]:
print(result["output"])

Las limitaciones de la cobertura de la póliza "PÓLIZA DE ACCIDENTES PERSONALES / REEMBOLSO GASTOS MÉDICOS" no están especificadas en la base de datos de pólizas de seguro. Te recomendaría revisar el documento de la póliza para obtener información detallada sobre las limitaciones de cobertura.


Let's test our agent with a question that is not in our dataset. So it's forced to use the Google Search API.


In [121]:
result = agent_executor(
    {
        "input": "puedes buscar en google cuales son las empresas de seguros en el pais de chile?"
    }
)



> Entering new AgentExecutor chain...



Invoking: `google_search` with `empresas de seguros en Chile`


Listado de Cías de Seguros Generales ; 99061000-2 · LIBERTY COMPAÑIA DE SEGUROS GENERALES S.A., VI ; 96508210-7 · MAPFRE COMPAÑIA DE SEGUROS GENERALES DE CHILE ... Apr 13, 2018 ... Las 10 mayores compañías de seguros generales de Chile por ganancias · N.° 1. La mayor compañía de seguros generales por ganancias en 2017 fue el ... Empresas. Brokers & Sponsors. #Protegiendotusdecisiones ... Solidez Financiera y Clasificaciones de Riesgo de Chubb Seguros Chile y Chubb Seguros de Vida Chile. Industria Aseguradora Chilena es elegida para liderar la Federación Interamericana de Empresas de Seguros ... Chile, Jorge Claude, fue electo presidente del ... Compra tu Seguro 100% online aquí: Seguro Automotriz, SOAP autos, SOAP Motos, SOAP Taxis, Asistencia en Viajes, Seguro de Hogar, entre otros ¡Más aquí! ... Seguros. Seguro Automotriz · Seguro de Hogar · Seguros para Empresas · SOAP · Otros Seguros · Consejos Liberty · Cybersegurida

In [122]:
print(result["output"])

Algunas de las empresas de seguros en Chile son:

1. Liberty Compañía de Seguros Generales S.A.
2. Mapfre Compañía de Seguros Generales de Chile
3. Chubb Seguros Chile
4. Zurich Chile Seguros de Vida S.A.
5. BCI Seguros
6. BNP Paribas Cardif Seguros Generales
7. Chilena Consolidada Seguros Generales

Este es solo un ejemplo de algunas de las empresas de seguros en Chile. Hay otras compañías de seguros disponibles en el país.


Let's test the Agent's memory by asking a follow-up question.


In [123]:
result = agent_executor(
    {"input": "cual fue mi primera pregunta y que respondiste en pocas palabras?"}
)



> Entering new AgentExecutor chain...


Tu primera pregunta fue: "¿Cuáles son las limitaciones de la cobertura de la póliza PÓLIZA DE ACCIDENTES PERSONALES / REEMBOLSO GASTOS MÉDICOS?" 
Mi respuesta fue: "Las limitaciones de la cobertura de esa póliza no están especificadas en la base de datos de pólizas de seguro. Te recomendaría revisar el documento de la póliza para obtener información detallada sobre las limitaciones de cobertura."

> Finished chain.


In [124]:
print(result["output"])

Tu primera pregunta fue: "¿Cuáles son las limitaciones de la cobertura de la póliza PÓLIZA DE ACCIDENTES PERSONALES / REEMBOLSO GASTOS MÉDICOS?" 
Mi respuesta fue: "Las limitaciones de la cobertura de esa póliza no están especificadas en la base de datos de pólizas de seguro. Te recomendaría revisar el documento de la póliza para obtener información detallada sobre las limitaciones de cobertura."


## Testing - Model Evaluation

Finally, we're ready to test our retriever. We will use our own `CustomQAGenerateChain` class to test our ``retriever_sqr`` tool. This function will take care of generating the questions and answers and evaluating them against a set of test questions.

To Summarize:

- We will generate an `eval_template` prompt which will be used to generate the questions and answers.
- Then, We will define a custom chain class `CustomQAGenerateChain` which will be defined for the specific task of generating questions and answers pairs.
- Also, We will Apply and Parse the data which will be stored in `exampleQA` variable.
- Finally, We will generate predictions with our `retriever_sqr` and evaluate them against the test questions using `QAEvalChain`.


In [125]:
eval_template = """Eres un profesor y estas conformando preguntas para hacer en un cuestionario.
Dado el siguiente documento, genere una pregunta y una respuesta basada en ese documento.

Ejemplo de formato del cuestionario:
<Documento Inicio>
...
<Documento Fin>
PREGUNTA: Aqui va la pregunta
RESPUESTA: Aqui va la respuesta

Estas preguntas deben ser detalladas y basarse explícitamente en la información del documento. Comencemos!

<Documento Inicio>
{doc}
<Documento Fin>"""

In [159]:
output_parser = RegexParser(
    regex=r"PREGUNTA: (.*?)\n+RESPUESTA: (.*)", output_keys=["query", "answer"]
)

EVAL_PROMPT = PromptTemplate(
    input_variables=["doc"], template=eval_template, output_parser=output_parser
)

In [160]:
custom_test_chain = CustomQAGenerateChain.from_llm(llm=llm, eval_prompt=EVAL_PROMPT)

qa_chunk_set = (
    r_splitter_result[7:9] + r_splitter_result[53:55] + r_splitter_result[100:102]
)

examplesQA = custom_test_chain.apply_and_parse(
    [{"doc": item.page_content} for item in qa_chunk_set]
)

/home/gio/ANYONEAI/InsurancePolicyChatbot/.venv/lib/python3.10/site-packages/langchain/chains/llm.py:308: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


Let's see what set of questions we have in our test set.


In [161]:
for item in examplesQA:
    print(item)

{'query': '¿Qué tipo de tratamiento cubre la cirugía dental por accidente?', 'answer': 'La cirugía dental por accidente cubre el tratamiento de lesiones a los dientes naturales, incluyendo exámenes dentales, extracciones, empastes, tratamiento dental en general y el reemplazo de las piezas dentales accidentadas.'}
{'query': '¿Qué es el servicio privado de enfermería y en qué condiciones se otorga?', 'answer': 'El servicio privado de enfermería es el servicio otorgado por un profesional de enfermería durante la hospitalización, siempre que haya sido prescrito por el médico tratante.'}
{'query': '¿Qué gastos médicos y ortopédicos están cubiertos por la póliza?', 'answer': 'La póliza cubre los gastos de aparatos auditivos, lentes o anteojos ópticos y de contacto, prótesis, órtesis, miembros artificiales, suministro de aparatos o equipos médicos y/u ortopédicos, así como también la adquisición o arriendo de equipos como sillas de ruedas, camas médicas, ventiladores mecánicos, entre otros.'

Now, let's predict the answers to our test questions.

In [162]:
predictions = qa_chain_mr.apply(examplesQA)

/home/gio/ANYONEAI/InsurancePolicyChatbot/.venv/lib/python3.10/site-packages/langchain/chains/llm.py:280: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


query='cirugía dental por accidente' filter=None limit=None
query='servicio privado de enfermería' filter=None limit=None
query='gastos médicos ortopédicos' filter=None limit=None
query='Accidentes del Trabajo y Enfermedades Profesionales' filter=None limit=None
query='requisitos médico especialista' filter=None limit=None
query='estado vegetativo persistente' filter=None limit=None


The prediction set is ready to be evaluated.

In [163]:
eval_chain = QAEvalChain.from_llm(llm=llm)
graded_output = eval_chain.evaluate(examplesQA, predictions)

Let's evaluate our predictions.

In [166]:
correct_count = 0

for i, eg in enumerate(examplesQA):
    print(f"Example {i}:")
    print("Question: " + predictions[i]["query"])
    print("Real Answer: " + predictions[i]["answer"])
    print("Predicted Answer: " + predictions[i]["result"])
    print("Predicted Grade: " + graded_output[i]["results"])
    correct_count += 1 if graded_output[i]["results"] == "CORRECT" else 0
    print()

print(
    f"Correctly answered {correct_count} out of {len(examplesQA)} questions. With a score of of {correct_count/len(examplesQA)*100}% "
)

Example 0:
Question: ¿Qué tipo de tratamiento cubre la cirugía dental por accidente?
Real Answer: La cirugía dental por accidente cubre el tratamiento de lesiones a los dientes naturales, incluyendo exámenes dentales, extracciones, empastes, tratamiento dental en general y el reemplazo de las piezas dentales accidentadas.
Predicted Answer: La cirugía dental por accidente cubre el tratamiento de lesiones a los dientes naturales causadas por un accidente, incluyendo exámenes dentales, extracciones, empastes, tratamiento dental en general y el reemplazo de las piezas dentales accidentadas. ¡Gracias por preguntar!
Predicted Grade: CORRECT

Example 1:
Question: ¿Qué es el servicio privado de enfermería y en qué condiciones se otorga?
Real Answer: El servicio privado de enfermería es el servicio otorgado por un profesional de enfermería durante la hospitalización, siempre que haya sido prescrito por el médico tratante.
Predicted Answer: El servicio privado de enfermería es un servicio otorga